# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [197]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [198]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [199]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [200]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [201]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['db'])
# cluster = Cluster()


# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [202]:
# TO-DO: Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
  WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 1 
  };
""")

#### Set Keyspace

In [203]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [204]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history
# that was heard during sessionId = 338, and itemInSession = 4

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
class ColumnNames:
    artist="artist"
    first_name="firstName"
    last_name="lastName"
    gender="gender"
    item_in_session="itemInSession"
    length="length"
    level="level"
    location="location"
    session_id="sessionId"
    user_id="userId"
    song="song"
    
    
session.execute("""
DROP TABLE IF EXISTS music_app_history
""")
session.execute("""
CREATE TABLE music_app_history(
   artist text, 
   title text, 
   length float,
   session_id bigint,
   item_in_session int,
   PRIMARY KEY (session_id,item_in_session)
   )
""")
query = session.prepare("""
INSERT INTO music_app_history(artist, title, length, session_id, item_in_session) VALUES (?,?,?,?,?)
""")

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        session.execute(query, (line[ColumnNames.artist],
                                line[ColumnNames.song],
                                float(line[ColumnNames.length]),
                                int(line[ColumnNames.session_id]),
                                int(line[ColumnNames.item_in_session])
                               ))

#### Do a SELECT to verify that the data have been inserted into each table

In [205]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history
# that was heard during sessionId = 338, and itemInSession = 4
r=session.execute("""
SELECT artist, title, length FROM music_app_history
WHERE session_id = 338 And item_in_session = 4
""")
for x in r:
    print(x)

Row(artist='Faithless', title='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [206]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
file = 'event_datafile_new.csv'

session.execute("""
DROP TABLE IF EXISTS artist_listening_history
""")
session.execute("""
CREATE TABLE artist_listening_history(
   artist text, 
   title text, 
   length float,
   username text,
   session_id int,
   item_in_session int,
   user_id int,
   PRIMARY KEY ((session_id,user_id),item_in_session)
   );
""")
query = session.prepare( """
INSERT INTO artist_listening_history(artist, title, length, username,session_id, item_in_session, user_id)
VALUES (?,?,?,?,?,?,?)
""")

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        values=(line[ColumnNames.artist], 
                line[ColumnNames.song],
                float(line[ColumnNames.length]),
                line[ColumnNames.first_name]+" "+line[ColumnNames.last_name],
                int(line[ColumnNames.session_id]),
                int(line[ColumnNames.item_in_session]),
                int(line[ColumnNames.user_id]))
#         print(values)
        session.execute(query, values ) 
                    

In [207]:
 ## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession)
#     and user (first and last name)\
## for userid = 10, sessionid = 182

r=session.execute("""

 SELECT artist, title, length FROM music_app_history
 WHERE session_id = 338 And item_in_session = 4
""")

for x in r:
    print(x)

Row(artist='Faithless', title='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [209]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened 
# to the song 'All Hands Against His Own'
file = 'event_datafile_new.csv'

session.execute("""
DROP TABLE IF EXISTS user_listening_history
""")
session.execute("""
CREATE TABLE user_listening_history(
   username text,
   song text,
   PRIMARY KEY (song)
   );
""")
query = session.prepare( """
INSERT INTO user_listening_history(username,song) VALUES (?,?)
""")

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        values=(line[ColumnNames.first_name]+" "+line[ColumnNames.last_name],
                line[ColumnNames.song])
#         print(values)
        session.execute(query, values ) 

In [210]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened 
# to the song 'All Hands Against His Own'

r=session.execute("""
SELECT username FROM user_listening_history
WHERE song='All Hands Against His Own'
""")
for x in r:
    print(x)

                    

Row(username='Sara Johnson')


### Drop the tables before closing out the sessions

In [193]:
## TO-DO: Drop the table before closing out the sessions

In [194]:
session.execute("""
DROP TABLE IF EXISTS sparkify.music_app_history
""")
session.execute("""
DROP TABLE IF EXISTS sparkify.artist_listening_history
""")
session.execute("""
DROP TABLE IF EXISTS sparkify.user_listening_history
""")

### Close the session and cluster connection¶

In [195]:
session.shutdown()
cluster.shutdown()